# Perform FFT analysis of the recorded waveforms
This notebook performs the discrete fourier analysis of the source audio files. The results are stored in a Pandas DataFrame with a response column added. This response is calculated to label observations as positive or negative for the presence of a beep.

Author: Reece Robinson, 2016

In [13]:
%matplotlib inline
import seaborn
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import os
import sys

from scipy.fftpack import rfft
from scipy.signal import blackman

In [39]:
source_directory = r'/Volumes/ThorsHammer/Data Science/data/audio-recognition/parts/'
destination_directory = r'/Volumes/ThorsHammer/Data Science/data/audio-recognition/fft/'

if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

# Create Dataset
### Calculate the FFT for the entire signal in 150 msec sliding window of 5 msec increments
The result dataset is a **n** x **m** dataframe where **n** is the number of observations and **m** is the calculated frequency power output from the FFT algorithm.

In [40]:
duration = 0.15 # window length in seconds
sample_rate = 16000
sample_size = 2400 # 150 ms window size
stride = 80 # sliding 5 ms window increments
w = blackman(sample_size) # FFT window used in analysis
label_window_start = 1000
label_window_end = 1300

In [41]:
from os.path import isfile, join

def list_files(base_dir):
    onlyfiles = [f for f in os.listdir(base_dir) if isfile(join(base_dir, f)) and f.split('.')[-1] == 'wav']
    return onlyfiles

files = list_files(source_directory)
print(files)


['161225-000_16bit-part-0.wav', '161225-000_16bit-part-1.wav', '161225-000_16bit-part-2.wav', '161225-000_16bit-part-3.wav', '161225-000_16bit-part-4.wav', '161225-000_16bit-part-5.wav', '161225-000_16bit-part-6.wav', '161225-001_16bit-part-0.wav', '161225-001_16bit-part-1.wav', '161225-001_16bit-part-10.wav', '161225-001_16bit-part-2.wav', '161225-001_16bit-part-3.wav', '161225-001_16bit-part-4.wav', '161225-001_16bit-part-5.wav', '161225-001_16bit-part-6.wav', '161225-001_16bit-part-7.wav', '161225-001_16bit-part-8.wav', '161225-001_16bit-part-9.wav', '161225-002_16bit-part-0.wav', '161225-003_16bit-part-0.wav', '161225-004_16bit-part-0.wav', '161225-005_16bit-part-0.wav', '161225-006_16bit-part-0.wav', '161225-006_16bit-part-1.wav', '161225-006_16bit-part-2.wav']


In [42]:
def analyse_ftt(filename, x_all):
    results = []
    
    # perform FFT for each sliding window
    for i in np.arange(0,len(x_all)-sample_size,stride):
        offset = i # seconds
        x = x_all[offset:offset+sample_size]

        FFTw = abs(rfft(x*w))
        FFTwdB = 20*scipy.log10(FFTw)
        results.append(FFTwdB)

    results = np.asarray(results)
    print('\nresults: %s,%s'%(results.shape))
    return results

In [44]:
def save_wav(df, x_all):
    # # Output Training Data
    positive_label_data = df[df['response'] == True].drop('response',axis=1)
    print ('\nPositive Response: %s,%s'%(positive_label_data.shape))

    # Positive Samples
    w_size = stride
    for idx in positive_label_data.index:
        begin = idx * w_size 
        librosa.output.write_wav(destination_directory+filename+'positive_sample_%s.wav'%idx,x_all[begin:begin+sample_size], sample_rate)

In [46]:
for filename in files:
    print('\nAnalysing: {0}'.format(filename))
    x_all, sr = librosa.load(source_directory+filename, sr=sample_rate)
    result = analyse_ftt(filename,x_all)
    df = pd.DataFrame(result)
    output_file = '{0}{1}-fft.csv.gz'.format(destination_directory,filename)
    %%time df.to_csv(output_file, header=True, index=False, compression='gzip')


Analysing: 161225-000_16bit-part-0.wav

results: 11970,2400
CPU times: user 2min 40s, sys: 1.65 s, total: 2min 42s
Wall time: 2min 41s

Analysing: 161225-000_16bit-part-1.wav

results: 11970,2400
CPU times: user 2min 32s, sys: 2.23 s, total: 2min 34s
Wall time: 2min 33s

Analysing: 161225-000_16bit-part-2.wav

results: 11970,2400
CPU times: user 2min 17s, sys: 1.32 s, total: 2min 18s
Wall time: 2min 17s

Analysing: 161225-000_16bit-part-3.wav

results: 11970,2400
CPU times: user 2min 28s, sys: 1.69 s, total: 2min 29s
Wall time: 2min 29s

Analysing: 161225-000_16bit-part-4.wav

results: 11970,2400
CPU times: user 2min 14s, sys: 1.41 s, total: 2min 15s
Wall time: 2min 15s

Analysing: 161225-000_16bit-part-5.wav

results: 11970,2400
CPU times: user 2min 17s, sys: 2.08 s, total: 2min 19s
Wall time: 2min 18s

Analysing: 161225-000_16bit-part-6.wav

results: 6846,2400
CPU times: user 1min 34s, sys: 1.58 s, total: 1min 35s
Wall time: 1min 35s

Analysing: 161225-001_16bit-part-0.wav

results:

## Total Observations: 275,352